In [1]:
from dl.simple_cnn import SimpleCNN
from dl.shuffle_net import ShuffleNetV2
from dl.mobile_net import MobileNET
from dl.wide_resnet import WideResNet
from dl.capsule_net import CapsuleNet
from dl.callbacks.plot_loss import PlotLosses
from data.data_set import get_data
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, TerminateOnNaN, EarlyStopping, ModelCheckpoint, TensorBoard
from config import DL_MODELS_PATH, TB_LOGS_PATH
import os
import matplotlib.pyplot as plt
import numpy as np
import pathlib
from swa.tfkeras import SWA
from dl.callbacks.cyclical_lr import CyclicLR
from plotly.offline import init_notebook_mode
from visualization.visualize_history import plot_accuracy_and_loss

init_notebook_mode(connected=True)

2019-12-19 20:16:19,062 : INFO : Checking/creating directories...
2019-12-19 20:16:19,063 : INFO : Directories are set.


In [3]:
((X_train, y_train), (X_val, y_val)) = get_data()

In [4]:
data_generator = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range= .8,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = .1, # Randomly zoom image 
        width_shift_range= .1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range= .1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=.05,  # randomly flip images horizontally
        vertical_flip=False)  # randomly flip images vertically

In [5]:
def get_callbacks(model_name, use_cyclic_lr=False, plot=False):
    callbacks = [TerminateOnNaN(), EarlyStopping(monitor='val_accuracy', patience=5)]
    callbacks.append(ModelCheckpoint(os.path.join(DL_MODELS_PATH, model_name + '-{epoch:02d}-{val_accuracy:.2f}.hdf5'), monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max'))
    tb_path = os.path.join(TB_LOGS_PATH, model_name)
    pathlib.Path(tb_path).mkdir(parents=True, exist_ok=True)
    callbacks.append(TensorBoard(log_dir=tb_path))
    callbacks.append(SWA(start_epoch=5, lr_schedule='constant', swa_lr=0.001, verbose=1))
    if plot:
        callbacks.append(PlotLosses())
    if use_cyclic_lr:
        callbacks.append(CyclicLR(base_lr=0.001, max_lr=0.006, step_size=2000.))
    else:
        rlronp =  ReduceLROnPlateau(monitor='val_loss',
                                    patience=3,
                                    verbose=1,
                                    factor=0.5,
                                    min_lr=0.00005)
        
        callbacks.append(rlronp)
    return callbacks

In [6]:
epochs = 100
Learning_rate = 0.001
decay= 5 * Learning_rate / epochs
optimizers = {'adam': Adam(lr=Learning_rate, decay= 3 * Learning_rate / epochs),
              'rmsprop': RMSprop(lr=Learning_rate, rho=0.9, epsilon=1e-08, decay= 0)}

In [7]:
epochs = 50
histories = {}
models = {}

for opt_name, opt in optimizers.items():
    '''
    # Train capsule net
    model_name = f'capsnet_{opt_name}'
    model = WideResNet(opt)
    history = model.train(data_generator, X_train, y_train, X_val, y_val,
                          16, epochs, get_callbacks(model_name))
    histories[model_name] = history.history
    models[model_name] = model
    
    
    # Train wide resnet net
    model_name = f'wide_resnet_{opt_name}'
    model = WideResNet(opt)
    history = model.train(data_generator, X_train, y_train, X_val, y_val,
                              32, epochs, get_callbacks(model_name))
    histories[model_name] = history.history
    models[model_name] = model

    # Train mobile_net
    model_name = f'mobile_net_{opt_name}'
    model = MobileNET(opt)
    history = model.train(data_generator, X_train, y_train, X_val, y_val,
                            16, epochs, get_callbacks(model_name))
    histories[model_name] = history.history
    models[model_name] = model
    
    # Train a basic cnn
    model_name = f'simple_cnn_{opt_name}'
    model = SimpleCNN(opt)
    history = model.train(data_generator, X_train, y_train, X_val, y_val,
                            2048, epochs, get_callbacks(model_name))
    histories[model_name] = history.history
    models[model_name] = model
    '''
    # Train shufflenetv2
    model_name = f'shuffle_netv2_cylic_lr_{opt_name}'
    model = ShuffleNetV2(opt)
    history = model.train(data_generator, X_train, y_train, X_val, y_val,
                              256, epochs, get_callbacks(model_name, use_cyclic_lr=True))
    histories[model_name] = history.history
    models[model_name] = model

Model uses batch normalization. SWA will require last epoch to be a forward pass and will run with no learning rate
Epoch 1/50
234/234 [============================>.] - ETA: 0s - loss: 0.7957 - accuracy: 0.6998 - categorical_crossentropy: 0.7957 - categorical_accuracy: 0.6998
Epoch 00001: val_categorical_accuracy improved from -inf to 0.10000, saving model to /home/cenk/Research/pre-trained-models/cv/fashion_mnist/shuffle_netv2_cylic_lr_adam-01-0.10.hdf5
235/234 [==============================] - 70s 299ms/step - loss: 0.7946 - accuracy: 0.7002 - categorical_crossentropy: 0.7946 - categorical_accuracy: 0.7002 - val_loss: 2.3070 - val_accuracy: 0.1000 - val_categorical_crossentropy: 2.3067 - val_categorical_accuracy: 0.1000
Epoch 2/50
234/234 [============================>.] - ETA: 0s - loss: 0.5677 - accuracy: 0.7855 - categorical_crossentropy: 0.5673 - categorical_accuracy: 0.7855
Epoch 00002: val_categorical_accuracy improved from 0.10000 to 0.18350, saving model to /home/cenk/Resea

Epoch 15/50
234/234 [============================>.] - ETA: 0s - loss: 0.2859 - accuracy: 0.8919 - categorical_crossentropy: 0.2859 - categorical_accuracy: 0.8919
Epoch 00015: val_categorical_accuracy did not improve from 0.89640
235/234 [==============================] - 68s 288ms/step - loss: 0.2859 - accuracy: 0.8920 - categorical_crossentropy: 0.2858 - categorical_accuracy: 0.8920 - val_loss: 0.3651 - val_accuracy: 0.8577 - val_categorical_crossentropy: 0.3688 - val_categorical_accuracy: 0.8577
Epoch 16/50
234/234 [============================>.] - ETA: 0s - loss: 0.2775 - accuracy: 0.8958 - categorical_crossentropy: 0.2776 - categorical_accuracy: 0.8958
Epoch 00016: val_categorical_accuracy improved from 0.89640 to 0.89660, saving model to /home/cenk/Research/pre-trained-models/cv/fashion_mnist/shuffle_netv2_cylic_lr_adam-16-0.90.hdf5
235/234 [==============================] - 67s 286ms/step - loss: 0.2773 - accuracy: 0.8958 - categorical_crossentropy: 0.2775 - categorical_accurac

Epoch 7/50
234/234 [============================>.] - ETA: 0s - loss: 0.4395 - accuracy: 0.8388 - categorical_crossentropy: 0.4397 - categorical_accuracy: 0.8388
Epoch 00007: val_categorical_accuracy did not improve from 0.74950
235/234 [==============================] - 94s 401ms/step - loss: 0.4395 - accuracy: 0.8388 - categorical_crossentropy: 0.4396 - categorical_accuracy: 0.8388 - val_loss: 2.5607 - val_accuracy: 0.6555 - val_categorical_crossentropy: 2.5321 - val_categorical_accuracy: 0.6555
Epoch 8/50
234/234 [============================>.] - ETA: 0s - loss: 0.4176 - accuracy: 0.8434 - categorical_crossentropy: 0.4178 - categorical_accuracy: 0.8434
Epoch 00008: val_categorical_accuracy did not improve from 0.74950
235/234 [==============================] - 97s 411ms/step - loss: 0.4174 - accuracy: 0.8436 - categorical_crossentropy: 0.4176 - categorical_accuracy: 0.8436 - val_loss: 2.9961 - val_accuracy: 0.6860 - val_categorical_crossentropy: 2.9430 - val_categorical_accuracy: 0

234/234 [============================>.] - ETA: 0s - loss: 0.2898 - accuracy: 0.8938 - categorical_crossentropy: 0.2901 - categorical_accuracy: 0.8938
Epoch 00022: val_categorical_accuracy did not improve from 0.90600
235/234 [==============================] - 93s 395ms/step - loss: 0.2897 - accuracy: 0.8937 - categorical_crossentropy: 0.2901 - categorical_accuracy: 0.8937 - val_loss: 0.3024 - val_accuracy: 0.8884 - val_categorical_crossentropy: 0.3046 - val_categorical_accuracy: 0.8884
Epoch 23/50
234/234 [============================>.] - ETA: 0s - loss: 0.2960 - accuracy: 0.8899 - categorical_crossentropy: 0.2960 - categorical_accuracy: 0.8899
Epoch 00023: val_categorical_accuracy did not improve from 0.90600
235/234 [==============================] - 96s 407ms/step - loss: 0.2961 - accuracy: 0.8898 - categorical_crossentropy: 0.2961 - categorical_accuracy: 0.8898 - val_loss: 0.3639 - val_accuracy: 0.8709 - val_categorical_crossentropy: 0.3658 - val_categorical_accuracy: 0.8709


In [8]:
plot_accuracy_and_loss()

2019-12-19 21:25:07,196 : ERROR : 'Key epoch_accuracy was not found in Reservoir'
2019-12-19 21:25:07,211 : ERROR : 'Key epoch_accuracy was not found in Reservoir'
2019-12-19 21:25:07,223 : ERROR : 'Key epoch_accuracy was not found in Reservoir'
2019-12-19 21:25:07,235 : ERROR : 'Key epoch_accuracy was not found in Reservoir'
2019-12-19 21:25:07,242 : ERROR : 'Key epoch_accuracy was not found in Reservoir'
2019-12-19 21:25:07,253 : ERROR : 'Key epoch_accuracy was not found in Reservoir'
2019-12-19 21:25:07,263 : ERROR : 'Key epoch_accuracy was not found in Reservoir'
2019-12-19 21:25:07,269 : ERROR : 'Key epoch_accuracy was not found in Reservoir'
2019-12-19 21:25:07,280 : ERROR : 'Key epoch_accuracy was not found in Reservoir'
2019-12-19 21:25:07,287 : ERROR : 'Key epoch_accuracy was not found in Reservoir'
2019-12-19 21:25:07,290 : ERROR : 'Key epoch_accuracy was not found in Reservoir'
2019-12-19 21:25:07,300 : ERROR : 'Key epoch_accuracy was not found in Reservoir'
